# Tutorial 04: Spatial Fundamentals

**Series**: PanelBox - Fundamentals  
**Level**: Intermediate  
**Estimated Time**: 60-70 minutes  
**Prerequisites**: Tutorial 01 (Panel Data Structures) - Independent of Tutorials 02-03

## Learning Objectives

By the end of this tutorial, you will be able to:
- Understand the concept of spatial dependence in panel data
- Create spatial weight matrices (contiguity, distance, k-nearest neighbors)
- Row-normalize and spectrally-normalize weight matrices
- Visualize spatial connections
- Calculate spatial lags
- Measure spatial autocorrelation (Moran's I)
- Understand when to use spatial panel models

## Table of Contents
1. [What is Spatial Dependence?](#1-what-is-spatial-dependence)
2. [Spatial Weight Matrices](#2-spatial-weight-matrices)
3. [Contiguity-Based Weights](#3-contiguity-based-weights)
4. [Distance-Based Weights](#4-distance-based-weights)
5. [K-Nearest Neighbors Weights](#5-k-nearest-neighbors-weights)
6. [Matrix Normalization](#6-matrix-normalization)
7. [Spatial Lags and Autocorrelation](#7-spatial-lags-and-autocorrelation)
8. [Practical Exercises](#8-practical-exercises)
9. [Summary and Next Steps](#9-summary-and-next-steps)

---

In [ ]:
# Notebook metadata
__version__ = "1.0.0"
__last_updated__ = "2026-02-16"
__compatible_with__ = "PanelBox >= 0.1.0"

# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy.spatial import distance_matrix
from IPython.display import display, Markdown

# PanelBox library (development mode)
import sys
sys.path.append('/home/guhaase/projetos/panelbox')
import panelbox as pb

# Try to import spatial module
try:
    from panelbox.spatial import SpatialWeights
    SPATIAL_AVAILABLE = True
except ImportError:
    SPATIAL_AVAILABLE = False
    print("Note: panelbox.spatial not fully implemented yet")
    print("Using manual implementations for demonstration")

# Plotting configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 8)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

# Display library version
print(f"PanelBox version: {pb.__version__}")
print(f"Notebook version: {__version__}")
print("Setup complete!")

## 1. What is Spatial Dependence?

### Tobler's First Law of Geography

> **"Everything is related to everything else, but near things are more related than distant things."**  
> — Waldo Tobler (1970)

### Spatial Dependence in Economics

**Spatial dependence** occurs when the value of a variable in one location depends on its values in neighboring locations.

#### Examples:

1. **Regional unemployment**: High unemployment in one region spills over to neighbors
   - Job seekers cross borders
   - Regional economic shocks spread

2. **House prices**: Property values correlated with neighborhood values
   - Similar amenities nearby
   - Comparable sales affect appraisals

3. **Trade flows**: Bilateral trade depends on third-country links
   - Trade partners of partners matter
   - Regional trade agreements create clusters

4. **Technology diffusion**: Innovation spreads geographically
   - Knowledge spillovers decay with distance
   - R&D centers create local externalities

---

### Why Standard Panel Methods Fail

**Traditional panel models assume**:
$$
Y_{it} = \beta X_{it} + \alpha_i + \lambda_t + \varepsilon_{it}
$$

Where errors are independent across entities.

**Problem with spatial data**:
$$
\text{Cov}(\varepsilon_{it}, \varepsilon_{jt}) \neq 0 \text{ for spatially connected } i, j
$$

**Consequences**:
- ❌ Biased coefficient estimates (if spatial lag omitted)
- ❌ Inconsistent standard errors (spatial correlation not accounted for)
- ❌ Inefficient estimates (spatial structure ignored)

**Solution**: Spatial panel models that explicitly model spatial dependence.

---

### Types of Spatial Models

1. **Spatial Lag Model (SAR)**:
   $$
   Y = \rho W Y + X\beta + \varepsilon
   $$
   Outcome depends on neighbors' outcomes (endogenous spatial effect)

2. **Spatial Error Model (SEM)**:
   $$
   Y = X\beta + u, \quad u = \lambda W u + \varepsilon
   $$
   Errors are spatially correlated (exogenous spatial effect)

3. **Spatial Durbin Model (SDM)**:
   $$
   Y = \rho W Y + X\beta + WX\gamma + \varepsilon
   $$
   Both outcome and predictors have spatial lags

**Foundation for all**: The **spatial weight matrix** $W$

---

## 2. Spatial Weight Matrices

### What is a Spatial Weight Matrix?

A **spatial weight matrix** $W$ (N×N) captures the spatial structure:
- $w_{ij}$ = strength of connection between entities $i$ and $j$
- $w_{ii} = 0$ (no self-connection)
- Rows correspond to "origin" entities
- Columns correspond to "destination" entities

**Interpretation**:
- $w_{ij} > 0$: Entities $i$ and $j$ are neighbors (connected)
- $w_{ij} = 0$: Entities $i$ and $j$ are not neighbors

---

### Example: 3 Regions

Suppose we have 3 regions: A, B, C
- A and B share a border (neighbors)
- B and C share a border (neighbors)
- A and C do not share a border (not neighbors)

**Binary contiguity matrix**:
$$
W = \begin{bmatrix}
0 & 1 & 0 \\
1 & 0 & 1 \\
0 & 1 & 0
\end{bmatrix}
$$

Row 1: A is connected to B (but not C)  
Row 2: B is connected to both A and C  
Row 3: C is connected to B (but not A)

---

### Types of Weight Matrices

| Type | Definition | Use Case |
|------|------------|----------|
| **Contiguity** | $w_{ij} = 1$ if $i$ and $j$ share border | Political units (countries, states) |
| **Distance** | $w_{ij} = 1/d_{ij}$ or $1/d_{ij}^2$ | Economic distance, gravity models |
| **K-nearest neighbors (KNN)** | $w_{ij} = 1$ if $j$ in $k$ nearest to $i$ | Uneven spatial distribution |
| **Economic** | $w_{ij} = $ trade share, similarity | Networks, trade flows |

---

In [ ]:
# Create synthetic spatial data (grid)
print("="*70)
print("CREATING SYNTHETIC SPATIAL DATA")
print("="*70)

# Create a 5x5 grid of regions
n_rows, n_cols = 5, 5
n_regions = n_rows * n_cols

# Generate coordinates
coords = []
region_ids = []
for i in range(n_rows):
    for j in range(n_cols):
        coords.append([j, i])  # (x, y)
        region_ids.append(f"R{i*n_cols + j + 1:02d}")

coords = np.array(coords)
spatial_data = pd.DataFrame({
    'region_id': region_ids,
    'x': coords[:, 0],
    'y': coords[:, 1]
})

# Add synthetic variable (e.g., GDP per capita with spatial pattern)
np.random.seed(42)
# Create spatial pattern: higher values in center
center_x, center_y = n_cols / 2, n_rows / 2
distance_from_center = np.sqrt((spatial_data['x'] - center_x)**2 +
                                (spatial_data['y'] - center_y)**2)
spatial_data['gdp_pc'] = 100 - 5 * distance_from_center + np.random.normal(0, 5, n_regions)

print(f"\nCreated {n_regions} regions in a {n_rows}×{n_cols} grid")
print("\nFirst 10 regions:")
display(spatial_data.head(10))

# Visualize the grid
fig, ax = plt.subplots(figsize=(8, 8))
scatter = ax.scatter(spatial_data['x'], spatial_data['y'],
                     c=spatial_data['gdp_pc'], s=300,
                     cmap='RdYlGn', edgecolors='black', linewidth=2)
for idx, row in spatial_data.iterrows():
    ax.text(row['x'], row['y'], row['region_id'],
            ha='center', va='center', fontsize=8, fontweight='bold')

ax.set_xlabel('X Coordinate', fontsize=12, fontweight='bold')
ax.set_ylabel('Y Coordinate', fontsize=12, fontweight='bold')
ax.set_title('Spatial Grid: 25 Regions', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.colorbar(scatter, label='GDP per capita', ax=ax)
plt.tight_layout()
plt.show()

print("\nNote: GDP is higher in the center (spatial pattern)")

## 3. Contiguity-Based Weights

### What is Contiguity?

**Contiguity**: Two regions are neighbors if they share a physical boundary.

#### Types:

1. **Rook contiguity** (4-neighbor): Share an edge
   - Like chess rook moves: up, down, left, right
   - Most common in regular grids

2. **Queen contiguity** (8-neighbor): Share an edge or corner
   - Like chess queen moves: includes diagonals
   - More inclusive definition
3. **Bishop contiguity** (4-diagonal): Share only corners
   - Rare in practice

---

### Rook vs Queen Example

Consider a 3×3 grid, focus on center cell (E):

```
A  B  C
D  E  F
G  H  I
```

- **Rook neighbors of E**: B, D, F, H (4 neighbors)
- **Queen neighbors of E**: A, B, C, D, F, G, H, I (8 neighbors)

---

In [ ]:
# Create rook contiguity matrix
print("="*70)
print("ROOK CONTIGUITY WEIGHT MATRIX")
print("="*70)

# Function to create rook contiguity matrix
def create_rook_contiguity(n_rows, n_cols):
    n = n_rows * n_cols
    W = np.zeros((n, n))

    for i in range(n):
        row_i = i // n_cols
        col_i = i % n_cols

        # Check 4 potential neighbors (up, down, left, right)
        neighbors = [
            (row_i - 1, col_i),  # up
            (row_i + 1, col_i),  # down
            (row_i, col_i - 1),  # left
            (row_i, col_i + 1)   # right
        ]

        for row_j, col_j in neighbors:
            if 0 <= row_j < n_rows and 0 <= col_j < n_cols:
                j = row_j * n_cols + col_j
                W[i, j] = 1

    return W

W_rook = create_rook_contiguity(n_rows, n_cols)

print(f"\nRook contiguity matrix: {W_rook.shape}")
print(f"Total connections: {int(W_rook.sum())}")
print(f"Average neighbors per region: {W_rook.sum(axis=1).mean():.2f}")

# Display subset
print("\nFirst 5×5 block of W (first 5 regions):")
display(pd.DataFrame(W_rook[:5, :5],
                     index=spatial_data['region_id'][:5],
                     columns=spatial_data['region_id'][:5]))

# Histogram of neighbor counts
neighbor_counts = W_rook.sum(axis=1)
plt.figure(figsize=(8, 5))
plt.hist(neighbor_counts, bins=np.arange(1, neighbor_counts.max()+2)-0.5,
         edgecolor='black', alpha=0.7, color='steelblue')
plt.xlabel('Number of Neighbors', fontsize=12, fontweight='bold')
plt.ylabel('Frequency', fontsize=12, fontweight='bold')
plt.title('Distribution of Neighbors (Rook Contiguity)', fontsize=14, fontweight='bold')
plt.xticks(range(int(neighbor_counts.max())+1))
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\nNote: Corner regions have 2 neighbors, edge regions 3, interior regions 4")

In [ ]:
# Create queen contiguity matrix
print("\n" + "="*70)
print("QUEEN CONTIGUITY WEIGHT MATRIX")
print("="*70)

def create_queen_contiguity(n_rows, n_cols):
    n = n_rows * n_cols
    W = np.zeros((n, n))

    for i in range(n):
        row_i = i // n_cols
        col_i = i % n_cols

        # Check 8 potential neighbors (including diagonals)
        for dr in [-1, 0, 1]:
            for dc in [-1, 0, 1]:
                if dr == 0 and dc == 0:
                    continue  # Skip self

                row_j = row_i + dr
                col_j = col_i + dc

                if 0 <= row_j < n_rows and 0 <= col_j < n_cols:
                    j = row_j * n_cols + col_j
                    W[i, j] = 1

    return W

W_queen = create_queen_contiguity(n_rows, n_cols)

print(f"\nQueen contiguity matrix: {W_queen.shape}")
print(f"Total connections: {int(W_queen.sum())}")
print(f"Average neighbors per region: {W_queen.sum(axis=1).mean():.2f}")

# Compare with rook
print("\n" + "-"*70)
print("ROOK vs QUEEN COMPARISON")
print("-"*70)
print(f"Rook connections: {int(W_rook.sum())}")
print(f"Queen connections: {int(W_queen.sum())}")
print(f"Additional diagonal connections: {int(W_queen.sum() - W_rook.sum())}")

# Visualize connections for a central region
central_idx = 12  # Center of 5×5 grid (R13)
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, W, title in zip(axes, [W_rook, W_queen], ['Rook', 'Queen']):
    ax.scatter(spatial_data['x'], spatial_data['y'], s=200,
               c='lightgray', edgecolors='black', linewidth=2, zorder=2)

    # Highlight central region
    ax.scatter(spatial_data.loc[central_idx, 'x'],
               spatial_data.loc[central_idx, 'y'],
               s=300, c='red', edgecolors='black', linewidth=2, zorder=3,
               label='Focal region')

    # Draw connections
    for j in range(n_regions):
        if W[central_idx, j] == 1:
            ax.plot([spatial_data.loc[central_idx, 'x'], spatial_data.loc[j, 'x']],
                    [spatial_data.loc[central_idx, 'y'], spatial_data.loc[j, 'y']],
                    'b-', linewidth=2, alpha=0.6, zorder=1)
            ax.scatter(spatial_data.loc[j, 'x'], spatial_data.loc[j, 'y'],
                       s=200, c='lightblue', edgecolors='black', linewidth=2, zorder=2)

    ax.set_xlabel('X Coordinate', fontsize=11, fontweight='bold')
    ax.set_ylabel('Y Coordinate', fontsize=11, fontweight='bold')
    ax.set_title(f'{title} Neighbors of {spatial_data.loc[central_idx, "region_id"]}',
                 fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Distance-Based Weights

### Distance Decay

**Principle**: Spatial interaction decreases with distance.

**Common formulations**:

1. **Inverse distance**:
   $$
   w_{ij} = \frac{1}{d_{ij}} \quad \text{for } d_{ij} > 0
   $$

2. **Inverse squared distance** (gravity model):
   $$
   w_{ij} = \frac{1}{d_{ij}^2}
   $$

3. **Exponential decay**:
   $$
   w_{ij} = \exp(-\alpha \cdot d_{ij})
   $$

4. **Threshold (cutoff)**:
   $$
   w_{ij} = \begin{cases}
   1 & \text{if } d_{ij} \leq d^* \\
   0 & \text{otherwise}
   \end{cases}
   $$

---

### When to Use Distance-Based Weights?

✅ **Good for**:
- Continuous space (not just discrete boundaries)
- Economic distance (not just physical)
- Gravity-type models (trade, migration)

❌ **Challenges**:
- Requires coordinate data
- Sensitive to distance metric choice
- Can create dense matrices (all pairs connected)

---

In [ ]:
# Calculate distance-based weights
print("="*70)
print("DISTANCE-BASED WEIGHT MATRICES")
print("="*70)

# Calculate Euclidean distances
coords_matrix = spatial_data[['x', 'y']].values
D = distance_matrix(coords_matrix, coords_matrix)

print(f"\nDistance matrix shape: {D.shape}")
print(f"Min distance (excluding diagonal): {D[D > 0].min():.4f}")
print(f"Max distance: {D.max():.4f}")

print("\nDistance matrix (first 5×5):")
display(pd.DataFrame(D[:5, :5],
                     index=spatial_data['region_id'][:5],
                     columns=spatial_data['region_id'][:5]))

# Create inverse distance matrix
W_inv_dist = 1 / (D + np.eye(n_regions))  # Add identity to avoid division by zero
np.fill_diagonal(W_inv_dist, 0)  # Set diagonal to 0

print("\n" + "-"*70)
print("INVERSE DISTANCE WEIGHTS")
print("-"*70)
print(f"W (inverse distance) shape: {W_inv_dist.shape}")
print("\nInverse distance matrix (first 5×5):")
display(pd.DataFrame(W_inv_dist[:5, :5],
                     index=spatial_data['region_id'][:5],
                     columns=spatial_data['region_id'][:5]))

# Visualize distance decay
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distance histogram
axes[0].hist(D[np.triu_indices_from(D, k=1)], bins=30,
             edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_xlabel('Distance', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[0].set_title('Distribution of Pairwise Distances', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Distance decay curve
distances = np.linspace(0.1, D.max(), 100)
axes[1].plot(distances, 1/distances, 'b-', linewidth=2, label='1/d')
axes[1].plot(distances, 1/distances**2, 'r--', linewidth=2, label='1/d²')
axes[1].plot(distances, np.exp(-0.5*distances), 'g:', linewidth=2, label='exp(-0.5d)')
axes[1].set_xlabel('Distance', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Weight', fontsize=12, fontweight='bold')
axes[1].set_title('Distance Decay Functions', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Distance threshold (cutoff)
print("\n" + "="*70)
print("DISTANCE THRESHOLD WEIGHTS")
print("="*70)

# Set threshold at median distance to immediate neighbors
threshold = 1.5  # In our grid, neighbors are at distance 1 or sqrt(2)

W_threshold = (D <= threshold).astype(float)
np.fill_diagonal(W_threshold, 0)

print(f"\nDistance threshold: {threshold}")
print(f"Connections: {int(W_threshold.sum())}")
print(f"Average neighbors: {W_threshold.sum(axis=1).mean():.2f}")

print("\nThreshold weight matrix (first 5×5):")
display(pd.DataFrame(W_threshold[:5, :5],
                     index=spatial_data['region_id'][:5],
                     columns=spatial_data['region_id'][:5]))

print("\nNote: This is similar to queen contiguity for regular grids")

## 5. K-Nearest Neighbors Weights

### What is KNN?

**K-Nearest Neighbors (KNN)**: Each region has exactly $k$ neighbors (the $k$ closest regions).

**Properties**:
- ✅ Balanced connectivity: All regions have same # of neighbors
- ✅ Robust to uneven spatial distribution
- ❌ Asymmetric: $i$ may be neighbor of $j$, but $j$ not neighbor of $i$

**Choice of $k$**:
- Too small (k=1): Sparse, underestimates spatial dependence
- Too large (k→N): Dense, overestimates spatial dependence
- Rule of thumb: k = 4 to 10 for most applications

---

### Symmetrization

KNN weights are asymmetric. To make symmetric:

1. **Union**: $w_{ij} = 1$ if $i$ in KNN of $j$ OR $j$ in KNN of $i$
2. **Intersection**: $w_{ij} = 1$ if $i$ in KNN of $j$ AND $j$ in KNN of $i$
3. **Average**: $w_{ij} = (w_{ij}^{KNN} + w_{ji}^{KNN})/2$

---

In [ ]:
# K-nearest neighbors weight matrix
print("="*70)
print("K-NEAREST NEIGHBORS WEIGHTS")
print("="*70)

k = 4  # Number of neighbors

def create_knn_weights(D, k):
    n = D.shape[0]
    W = np.zeros((n, n))

    for i in range(n):
        # Find k nearest neighbors (excluding self)
        distances = D[i, :]
        nearest_indices = np.argsort(distances)[1:k+1]  # Skip first (self)
        W[i, nearest_indices] = 1

    return W

W_knn = create_knn_weights(D, k)

print(f"\nK = {k}")
print(f"Total connections: {int(W_knn.sum())}")
print(f"Neighbors per region: {W_knn.sum(axis=1)}")
print(f"Is symmetric? {np.allclose(W_knn, W_knn.T)}")

print("\nKNN weight matrix (first 5×5):")
display(pd.DataFrame(W_knn[:5, :5],
                     index=spatial_data['region_id'][:5],
                     columns=spatial_data['region_id'][:5]))

# Check asymmetry
asymmetric_pairs = 0
for i in range(n_regions):
    for j in range(i+1, n_regions):
        if W_knn[i, j] != W_knn[j, i]:
            asymmetric_pairs += 1

print(f"\nAsymmetric pairs: {asymmetric_pairs}")

# Symmetrize (union method)
W_knn_sym = np.maximum(W_knn, W_knn.T)
print(f"\nSymmetrized (union) connections: {int(W_knn_sym.sum())}")
print(f"Is symmetric? {np.allclose(W_knn_sym, W_knn_sym.T)}")

# Visualize KNN connections
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, W, title in zip(axes,
                        [W_knn, W_knn_sym],
                        ['K-Nearest Neighbors (Original)', 'KNN Symmetrized (Union)']):
    # Plot all regions
    ax.scatter(spatial_data['x'], spatial_data['y'], s=150,
               c='lightgray', edgecolors='black', linewidth=1.5, zorder=2)

    # Draw connections
    for i in range(n_regions):
        for j in range(i+1, n_regions):
            if W[i, j] == 1 or W[j, i] == 1:
                ax.plot([spatial_data.loc[i, 'x'], spatial_data.loc[j, 'x']],
                        [spatial_data.loc[i, 'y'], spatial_data.loc[j, 'y']],
                        'b-', linewidth=0.8, alpha=0.4, zorder=1)

    ax.set_xlabel('X Coordinate', fontsize=11, fontweight='bold')
    ax.set_ylabel('Y Coordinate', fontsize=11, fontweight='bold')
    ax.set_title(f'{title} (k={k})', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Matrix Normalization

### Why Normalize?

**Raw weight matrices** have arbitrary scales. Normalization standardizes them for interpretation and stability.

### Row Normalization (Most Common)

Divide each row by its sum:
$$
w_{ij}^{row} = \frac{w_{ij}}{\sum_{k=1}^N w_{ik}}
$$

**Interpretation**: $w_{ij}^{row}$ = share of influence that $j$ has on $i$

**Properties**:
- ✅ Row sums = 1 (weights are proportions)
- ✅ Natural interpretation: "average of neighbors"
- ✅ Spatial lag is weighted average

---

### Spectral Normalization

Divide by maximum eigenvalue:
$$
W^{spectral} = \frac{W}{\lambda_{\max}(W)}
$$

**Purpose**: Ensures stability in spatial autoregressive models ($|\rho| < 1$ required)

**Properties**:
- ✅ Maximum eigenvalue = 1
- ✅ Mathematical convenience for estimation
- ❌ Less intuitive interpretation

---

### Comparison

| Normalization | Row Sums | Max Eigenvalue | Use Case |
|---------------|----------|----------------|----------|
| None (binary) | Varies | > 1 | Initial construction |
| Row | 1 | ≤ 1 | Most common (interpretation) |
| Spectral | Varies | 1 | Theoretical models |

---

In [ ]:
# Row normalization
print("="*70)
print("ROW NORMALIZATION")
print("="*70)

def row_normalize(W):
    """Row-normalize a weight matrix."""
    row_sums = W.sum(axis=1, keepdims=True)
    # Avoid division by zero (regions with no neighbors)
    row_sums[row_sums == 0] = 1
    return W / row_sums

W_rook_row = row_normalize(W_rook)

print("\nOriginal rook matrix (first 5×5):")
display(pd.DataFrame(W_rook[:5, :5],
                     index=spatial_data['region_id'][:5],
                     columns=spatial_data['region_id'][:5]))

print("\nRow-normalized rook matrix (first 5×5):")
display(pd.DataFrame(W_rook_row[:5, :5],
                     index=spatial_data['region_id'][:5],
                     columns=spatial_data['region_id'][:5]))

# Verify row sums
print("\n" + "-"*70)
print("VERIFICATION")
print("-"*70)
row_sums = W_rook_row.sum(axis=1)
print(f"Row sums (first 10): {row_sums[:10]}")
print(f"All row sums = 1? {np.allclose(row_sums, 1)}")

print("\nInterpretation:")
print("  - Each row sums to 1")
print("  - Each entry = fraction of neighbor's influence")
print("  - Example: If region has 4 neighbors, each gets weight 0.25")

In [ ]:
# Spectral normalization
print("\n" + "="*70)
print("SPECTRAL NORMALIZATION")
print("="*70)

def spectral_normalize(W):
    """Spectral normalization (divide by max eigenvalue)."""
    eigenvalues = np.linalg.eigvals(W)
    lambda_max = np.max(np.abs(eigenvalues))
    if lambda_max == 0:
        return W
    return W / lambda_max

W_rook_spectral = spectral_normalize(W_rook)

# Calculate eigenvalues
eig_original = np.linalg.eigvals(W_rook)
eig_spectral = np.linalg.eigvals(W_rook_spectral)

print("\nOriginal matrix:")
print(f"  Max eigenvalue: {np.max(np.abs(eig_original)):.4f}")

print("\nSpectral normalized matrix:")
print(f"  Max eigenvalue: {np.max(np.abs(eig_spectral)):.4f}")

# Compare normalizations
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

matrices = [W_rook, W_rook_row, W_rook_spectral]
titles = ['Original (Binary)', 'Row Normalized', 'Spectral Normalized']

for ax, W, title in zip(axes, matrices, titles):
    im = ax.imshow(W[:10, :10], cmap='Blues', vmin=0, vmax=1, aspect='auto')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_xlabel('Region j', fontsize=10)
    ax.set_ylabel('Region i', fontsize=10)
    plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

print("\nNote: Row normalization varies by row; spectral scales uniformly")

## 7. Spatial Lags and Autocorrelation

### What is a Spatial Lag?

The **spatial lag** of variable $Y$ is the weighted average of neighbors' values:
$$
(WY)_i = \sum_{j=1}^N w_{ij} Y_j
$$

**With row-normalized $W$**:
$$
(WY)_i = \text{average of } Y \text{ in neighbors of } i
$$

---

### Interpretation

**Example**: Regional unemployment
- $Y_i$ = unemployment rate in region $i$
- $(WY)_i$ = average unemployment rate in neighboring regions

**Spatial lag model**:
$$
Y_i = \rho (WY)_i + X_i \beta + \varepsilon_i
$$

Where $\rho$ = spatial autocorrelation parameter

---

### Moran's I (Global Spatial Autocorrelation)

**Moran's I** tests for spatial autocorrelation:
$$
I = \frac{N}{\sum_i \sum_j w_{ij}} \cdot \frac{\sum_i \sum_j w_{ij}(Y_i - \bar{Y})(Y_j - \bar{Y})}{\sum_i (Y_i - \bar{Y})^2}
$$

**Interpretation**:
- $I > 0$: Positive spatial autocorrelation (similar values cluster)
- $I ≈ 0$: No spatial autocorrelation (random)
- $I < 0$: Negative spatial autocorrelation (dissimilar values cluster)

**Range**: Approximately $[-1, 1]$

---

In [ ]:
# Calculate spatial lag
print("="*70)
print("SPATIAL LAG")
print("="*70)

# Use row-normalized rook weights
y = spatial_data['gdp_pc'].values
Wy = W_rook_row @ y  # Matrix multiplication

spatial_data['gdp_pc_lag'] = Wy

print("\nOriginal GDP per capita vs Spatial Lag:")
display(spatial_data[['region_id', 'gdp_pc', 'gdp_pc_lag']].head(10))

# Scatter plot: Y vs WY
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(y, Wy, alpha=0.7, edgecolors='k', linewidth=0.5, s=80)

# Add regression line
from scipy.stats import linregress
slope, intercept, r_value, p_value, std_err = linregress(y, Wy)
x_line = np.linspace(y.min(), y.max(), 100)
y_line = slope * x_line + intercept
ax.plot(x_line, y_line, 'r--', linewidth=2,
        label=f'Slope = {slope:.3f} (Moran\'s I proxy)')

ax.set_xlabel('GDP per capita (Y)', fontsize=12, fontweight='bold')
ax.set_ylabel('Spatial Lag (WY)', fontsize=12, fontweight='bold')
ax.set_title('Moran Scatterplot: Y vs WY', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nSlope of Y vs WY: {slope:.4f}")
print("  → Positive slope indicates positive spatial autocorrelation")
print("  → High GDP regions tend to have high-GDP neighbors")

In [ ]:
# Moran's I statistic
print("\n" + "="*70)
print("MORAN'S I TEST")
print("="*70)

def morans_i(y, W):
    """Calculate Moran's I statistic."""
    n = len(y)
    y_mean = y.mean()
    y_dev = y - y_mean

    # Numerator: sum of products of deviations weighted by W
    numerator = np.sum(W * np.outer(y_dev, y_dev))

    # Denominator: sum of squared deviations
    denominator = np.sum(y_dev**2)

    # Normalization factor
    S0 = W.sum()  # Sum of all weights

    I = (n / S0) * (numerator / denominator)

    return I

I_observed = morans_i(y, W_rook_row)

print(f"\nMoran's I statistic: {I_observed:.4f}")

# Expected value under null (no spatial autocorrelation)
n = len(y)
E_I = -1 / (n - 1)
print(f"Expected I (H₀): {E_I:.4f}")

# Interpretation
if I_observed > E_I:
    print(f"\nInterpretation:")
    print(f"  I = {I_observed:.4f} > E(I) = {E_I:.4f}")
    print(f"  → Positive spatial autocorrelation detected")
    print(f"  → GDP per capita exhibits spatial clustering")
    print(f"  → High-value regions near high-value regions (or vice versa)")
else:
    print(f"\nI ≈ {I_observed:.4f} ≤ E(I) = {E_I:.4f}")
    print(f"→ No strong spatial autocorrelation")

# Visualize spatial pattern
fig, ax = plt.subplots(figsize=(8, 8))
scatter = ax.scatter(spatial_data['x'], spatial_data['y'],
                     c=spatial_data['gdp_pc'], s=400,
                     cmap='RdYlGn', edgecolors='black', linewidth=2)
for idx, row in spatial_data.iterrows():
    ax.text(row['x'], row['y'], f"{row['gdp_pc']:.1f}",
            ha='center', va='center', fontsize=7, fontweight='bold')

ax.set_xlabel('X Coordinate', fontsize=12, fontweight='bold')
ax.set_ylabel('Y Coordinate', fontsize=12, fontweight='bold')
ax.set_title(f'Spatial Pattern of GDP per capita (Moran\'s I = {I_observed:.3f})',
             fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='GDP per capita', ax=ax)
plt.tight_layout()
plt.show()

print("\nNote: Visual inspection confirms clustering (high GDP in center)")

## 8. Practical Exercises

Solutions in `/solutions/01_fundamentals/04_spatial_solutions.ipynb`.

### Exercise 1: Compare Weight Matrices

**Task**: Compare rook, queen, and KNN (k=6) weight matrices.

1. Create all three matrices for our 5×5 grid
2. Calculate: total connections, average neighbors, density
3. Which is most connected? Least connected?
4. Visualize network structure for one focal region

---

### Exercise 2: Custom Distance Function

**Task**: Create a distance-based weight matrix with exponential decay.

1. Use formula: $w_{ij} = \exp(-0.3 \cdot d_{ij})$
2. Set $w_{ii} = 0$ (no self-connection)
3. Row-normalize the matrix
4. Compare with inverse distance weights

---

### Exercise 3: Spatial Lag with Real Data

**Task**: Use Grunfeld dataset to compute spatial lag of investment.

1. Assume firms can be ordered by firm ID
2. Create KNN weight matrix (k=3) based on value similarity
   - Distance = |value_i - value_j|
3. Compute spatial lag of investment
4. Plot investment vs its spatial lag

---

### Exercise 4: Test for Spatial Autocorrelation

**Task**: Test if investment in Grunfeld exhibits spatial autocorrelation.

1. Use firm averages across years (N=10)
2. Create contiguity matrix (assume firms 1-5 in one group, 6-10 in another)
3. Calculate Moran's I
4. Interpret: Is there spatial dependence?

---

In [ ]:
# Exercise 1: Your code here
# -------------------------



In [ ]:
# Exercise 2: Your code here
# -------------------------



In [ ]:
# Exercise 3: Your code here
# -------------------------



In [ ]:
# Exercise 4: Your code here
# -------------------------



## 9. Summary and Next Steps

### What You Learned

In this tutorial, you mastered:

✅ **Spatial dependence concept**: Tobler's Law, economic examples  
✅ **Weight matrices**: Contiguity, distance, KNN  
✅ **Matrix construction**: Rook, queen, inverse distance, threshold  
✅ **Normalization**: Row and spectral normalization  
✅ **Spatial lags**: Weighted averages of neighbors  
✅ **Spatial autocorrelation**: Moran's I statistic

---

### Key Formulas

| Concept | Formula | Interpretation |
|---------|---------|----------------|
| Weight matrix | $W$ (N×N) | Spatial connections |
| Contiguity | $w_{ij} = 1$ if adjacent | Binary neighbors |
| Inverse distance | $w_{ij} = 1/d_{ij}$ | Distance decay |
| Row normalization | $w_{ij}^{row} = w_{ij}/\sum_k w_{ik}$ | Share of influence |
| Spatial lag | $(WY)_i = \sum_j w_{ij}Y_j$ | Weighted average of neighbors |
| Moran's I | $I = \frac{N}{S_0}\frac{\sum_i\sum_j w_{ij}(Y_i-\bar{Y})(Y_j-\bar{Y})}{\sum_i(Y_i-\bar{Y})^2}$ | Global autocorrelation |

---

### When to Use Spatial Models?

Use spatial panel models when:
- ✅ Data has geographic structure (regions, countries, locations)
- ✅ Moran's I indicates significant spatial autocorrelation
- ✅ Economic theory suggests spillovers (trade, knowledge, policies)
- ✅ Residuals from standard models show spatial patterns

**Next**: Spatial panel estimators (Spatial Lag, Spatial Error, Spatial Durbin)

---

### Next Steps

**Option A: Continue to Module 2 - Classical Estimators**
- Fixed Effects, Random Effects (non-spatial)
- Most common path

**Option B: Jump to Module 4 - Spatial Panel Models**
- Spatial Lag Model (SAR)
- Spatial Error Model (SEM)
- Spatial Durbin Model (SDM)
- Requires: This tutorial + Tutorial 03 (Estimation)

**Recommended**: Option A (complete fundamentals first)

---

### Quick Reference: Weight Matrix Types

```python
# Contiguity (rook)
W_rook = create_rook_contiguity(n_rows, n_cols)

# Contiguity (queen)
W_queen = create_queen_contiguity(n_rows, n_cols)

# Distance (inverse)
D = distance_matrix(coords, coords)
W_dist = 1 / (D + np.eye(N))
np.fill_diagonal(W_dist, 0)

# K-Nearest Neighbors
W_knn = create_knn_weights(D, k=4)

# Normalization
W_row = row_normalize(W)
W_spectral = spectral_normalize(W)

# Spatial lag
Wy = W_row @ y
```

---

### Additional Resources

- **Anselin (1988)**: "Spatial Econometrics: Methods and Models" (classic text)
- **LeSage & Pace (2009)**: "Introduction to Spatial Econometrics" (comprehensive)
- **Elhorst (2014)**: "Spatial Econometrics: From Cross-Sectional Data to Spatial Panels" (panel focus)
- **PySAL**: Python Spatial Analysis Library (https://pysal.org)

---

In [ ]:
# Session information
print("="*70)
print("SESSION INFORMATION")
print("="*70)
print(f"\nNotebook: 04_spatial_fundamentals.ipynb")
print(f"Version: {__version__}")
print(f"Last updated: {__last_updated__}")
print(f"\nLibrary versions:")
print(f"  PanelBox: {pb.__version__}")
print(f"  NumPy: {np.__version__}")
print(f"  Pandas: {pd.__version__}")
print(f"  SciPy: {scipy.__version__}")
print("\nTutorial completed successfully! 🎉")
print("You now understand spatial weight matrices!")